In [1]:
!nvidia-smi

Sun Sep 15 13:01:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P3              20W /  55W |      0MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [65]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
import os
import json
from langchain.schema import Document

In [41]:
PINECONE_API_KEY="8b9ea68a-fcc2-4d84-989a-f6b35212f328"
PINECONE_API_ENV="us-east-1"

In [66]:
def convert_to_documents(documents):
    document_objects = []
    for doc in documents:
        # Use .get() to handle missing keys with default values
        product_info = f"""
        Product Name: {doc.get('product_name', 'N/A')}
        Price: {doc.get('product_price', 'N/A')}
        Description: {doc.get('product_description', 'N/A')}
        Artisan: {doc.get('artisan', 'N/A')}
        Category: {doc.get('product_category', 'N/A')} ({doc.get('product_sub_category', 'N/A')})
        Color: {doc.get('product_color', 'N/A')}
        Size: {doc.get('product_size', 'N/A')}
        Weight: {doc.get('product_weight', 'N/A')}
        Material: {doc.get('product_material', 'N/A')}
        Care Instructions: {doc.get('product_care', 'N/A')}
        Craft: {doc.get('product_craft', 'N/A')}
        """
        document_objects.append(Document(page_content=product_info.strip()))  # Create Document object
    return document_objects

In [62]:
def load_json(filepath):
    with open(filepath, 'r') as f:
        documents = json.load(f)
    return documents

In [63]:
extracted_data = load_json("data/updated_itokri_small_data.json")

In [67]:
document_objects = convert_to_documents(extracted_data)

In [68]:
#create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(document_objects)

    return text_chunks

In [70]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

258


In [71]:
#download embedding model
def hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
    return embedding

In [17]:
embedding = hugging_face_embedding()

d:\Anaconda\envs\quant-chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\Anaconda\envs\quant-chatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [72]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L12-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [73]:
os.environ['PINECONE_API_KEY'] = '8b9ea68a-fcc2-4d84-989a-f6b35212f328'
index_name = "vaani-chatbot"

docsearch=PineconeVectorStore.from_texts(
    [t.page_content for t in text_chunks],
    index_name=index_name,
    embedding=embedding
)

In [74]:
os.environ['PINECONE_API_KEY'] = '8b9ea68a-fcc2-4d84-989a-f6b35212f328'
index_name = "vaani-chatbot"

In [75]:
docsearch = Pinecone.from_existing_index(index_name,embedding)

query = "Beige - Kalamkari Block Printed Cotton Elasticated Pant"

docs=docsearch.similarity_search(query,k=2)

print(docs)

[Document(page_content="Product Name: Beige - Kalamkari Block Printed Cotton Elasticated Pant\n        Price: 1,390.00\n        Description: Beige - Kalamkari Block Printed Cotton Elasticated Pant. Elasticated pant with a beautiful kalamkari block print. The pant is made from high quality cotton and has a comfortable fit. It is perfect for casual wear and can be paired with a kurta or a t-shirt.\n        Artisan: Itokri Craft Initiative \n        Category: Women's Fashion (Pants)\n        Color: N/A\n        Size: XL"), Document(page_content="Product Name: Beige - Kalamkari Block Printed Cotton Kaftan With Tie-Up Waist (Medium)\n        Price: 1,490.00\n        Description: Beige - Kalamkari Block Printed Cotton Kaftan With Tie-Up Waist (Medium)\n        Artisan: Itokri Craft Initiative \n        Category: Women's Fashion (Kaftans)\n        Color: N/A\n        Size: Medium\n        Weight: \n        Material: Cotton\n        Care Instructions: Hand Washable\n        Craft: Kalamkari Bl

In [76]:
template="""[INST] 
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say you don't know, and don't try to make up an answer.
You are Vaani, an Indian handicraft e-commerce assistant, providing useful and concise information in two lines.

Context: {context}
Question: {question}

Only return a helpful answer below and nothing else.
Helpful answer:
[/INST]

"""

In [77]:
prompt=PromptTemplate(input_variables=["context","question"],template=template)

In [78]:
chain_type_kwargs={"prompt":prompt}

In [79]:
llm = CTransformers(model="D:\GenAi\quantum_computing_chatbot\model\llama-2-13b-chat.ggmlv3.q5_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':350,
                            'temperature':0.5}
                    )

In [80]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [81]:
user_input="i wanna a handicraft from assam?"

In [82]:
result=qa({"query":user_input})
print("Response: ",result["result"])

d:\Anaconda\envs\quant-chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Response:  Hello! If you're looking for a handicraft from Assam, I recommend the Assam Traditional Handloom Tussar Mekhela Chador With Blouse Piece 19. It's made of high-quality Tussar silk and features intricate Assamese weaving techniques. Plus, it's crafted by skilled artisan Sheikh Moidun Nilufar. Would you like to know more about this product?
